# 1. Parse Menu

In [2]:
import json

with open('menu.json') as f:
  data = json.load(f)
menu_data = []

for menu_category, menu_item_dict in data.items():
    for menu_item, menu_item_info in menu_item_dict.items():
        if menu_category != 'Menus' and isinstance(menu_item_info, list):
            nutritional_info = {}
            item_id = menu_item
            if menu_item_info[2].keys() == {'nutritionalInfo', 'available'}:
                nutritional_info = menu_item_info[2]['nutritionalInfo']
                nutritional_info.pop('itemId')
                available = menu_item_info[2]['available']
                
            elif menu_item_info[2].keys() == {'available'}:
                available = menu_item_info[2]['available']
                
            else:
                pass
            
            allergens = nutritional_info.get('allergens', [])
            nutritional_info.pop('allergens', None)
            result = {
                "itemId": item_id,
                "category": menu_category,
                "name": menu_item_info[0],
                "price": float(menu_item_info[1]),
                "soy" : (True if 'soy' in allergens else False),
                "lactose" : (True if ("dairy" in allergens) or ("whey" in allergens) else False),
                "gluten" : (True if 'wheat' in allergens else False),
                "nutritionalInfo": nutritional_info,
                "contents": [],
                "available": bool(available)
            }
            print(result)
            menu_data.append(result)
            
        elif menu_category == 'Menus':
            item_id = menu_item
            name = menu_item_info['name']
            price = menu_item_info['price']
            contents = menu_item_info['contents']
            
            new_contents = []
            for item in contents:
                if isinstance(item, list):
                    _item_id = item[0]
                    _qty = item[1]
                    new_contents.append({
                        "itemId": _item_id,
                        "quantity": _qty
                    })
                
                elif isinstance(item, dict):
                    new_contents.append(item)
                else:
                    continue
                
            allergens = nutritional_info.get('allergens', [])
            nutritional_info.pop('allergens', None)
            result = {
                "itemId": item_id,
                "category": menu_category,
                "name": name,
                "price": price,
                "soy" : (True if 'soy' in allergens else False),
                "lactose" : (True if ("dairy" in allergens) or ("whey" in allergens) else False),
                "gluten" : (True if 'wheat' in allergens else False),
                "nutritionalInfo": {},
                "contents": new_contents,
                "available": True
            }
            print(result)
            menu_data.append(result)

{'itemId': 'C1', 'category': 'Chicken', 'name': 'Original Recipe', 'price': 3.5, 'soy': True, 'lactose': False, 'gluten': True, 'nutritionalInfo': {'kcal': 400, 'fat': 22, 'protein': 28}, 'contents': [], 'available': False}
{'itemId': 'C2', 'category': 'Chicken', 'name': 'Popcorn Chicken', 'price': 4.0, 'soy': True, 'lactose': False, 'gluten': True, 'nutritionalInfo': {'kcal': 350, 'fat': 20, 'protein': 25}, 'contents': [], 'available': False}
{'itemId': 'C4', 'category': 'Chicken', 'name': 'Hot Wings', 'price': 3.0, 'soy': False, 'lactose': False, 'gluten': True, 'nutritionalInfo': {'kcal': 270, 'fat': 18, 'protein': 19}, 'contents': [], 'available': False}
{'itemId': 'C5', 'category': 'Chicken', 'name': 'Snackbox', 'price': 15.0, 'soy': False, 'lactose': False, 'gluten': False, 'nutritionalInfo': {'kcal': 150, 'fat': 100, 'protein': 10}, 'contents': [], 'available': False}
{'itemId': 'C6', 'category': 'Chicken', 'name': 'Crispy Tenders', 'price': 15.0, 'soy': False, 'lactose': False,

# 2. Upload to DB

In [10]:
import weaviate
import weaviate.classes as wvc
from weaviate.classes.config import DataType, Property, VectorDistances
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
load_dotenv()

client = weaviate.connect_to_local()
client.collections.delete("Kfc_menu")
embedding_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

In [11]:
client.collections.delete("Kfc_menu")
if not client.collections.exists("Kfc_menu"):
    collection = client.collections.create(
            name = "Kfc_menu",
            vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                distance_metric=VectorDistances.COSINE,
                quantizer = wvc.config.Configure.VectorIndex.Quantizer.pq()
            ),
            properties = [
                Property(
                    name = "itemId",
                    data_type = DataType.TEXT,
                    index_searchable=True,
                ),
                Property(
                    name = "category",
                    data_type = DataType.TEXT,
                    index_searchable=True,
                ),
                Property(
                    name = "name",
                    data_type = DataType.TEXT,
                    index_searchable=True,
                ),
                Property(
                    name = "price",
                    data_type = DataType.NUMBER,
                ),
                Property(
                    name = "nutritionalInfo",
                    data_type = DataType.OBJECT,
                    nested_properties=[
                        Property(
                            name = "kcal",
                            data_type = DataType.NUMBER,
                        ),
                        Property(
                            name = "fat",
                            data_type = DataType.NUMBER,
                            
                        ),
                        Property(
                            name = "protein",
                            data_type = DataType.NUMBER,
                        ),
                    ],
                    # skip_vectorization=True
                ),
                Property(
                    name = "lactose",
                    data_type = DataType.BOOL,
                    # index_searchable=True
                ),
                Property(
                    name = "gluten",
                    data_type = DataType.BOOL,
                    # index_searchable=True
                ),
                Property(
                    name = "soy",
                    data_type = DataType.BOOL,
                ),
                Property(
                    name="contents",
                    data_type=DataType.OBJECT_ARRAY,
                    skip_vectorization=True,
                    nested_properties=[
                        Property(
                            name="itemId",
                            data_type=DataType.TEXT
                        ),
                        Property(
                            name="quantity",
                            data_type=DataType.NUMBER
                        ),
                        Property(
                            name="from",
                            data_type=DataType.TEXT
                        ),
                        Property(
                            name="choose",
                            data_type=DataType.NUMBER
                        ),
                        Property(
                            name="size",
                            data_type=DataType.TEXT
                        )
                    ]
                ),
                Property(
                    name = "available",
                    data_type = DataType.BOOL,
                ),
            ]
        )
    
with collection.batch.dynamic() as batch:
    for item in menu_data:
        allergens_list = []
        if item["gluten"]:
            allergens_list.append('wheat (CONTAINS GLUTEN)')
        if item["lactose"]:
            allergens_list.append('dairy (CONTAINS LACTOSE)')
        if item["soy"]:
            allergens_list.append('soy (CONTAINS SOY)')
        else:
            pass

        string_to_embed = f"allergens:{allergens_list or ''}, name:{item['name'].lower()}, category:{item['category'].lower()}"
        print(string_to_embed)
        vector = embedding_model.encode(string_to_embed)
        batch.add_object(
            properties=item,
            vector = vector
        )

allergens:['wheat (CONTAINS GLUTEN)', 'soy (CONTAINS SOY)'], name:original recipe, category:chicken
allergens:['wheat (CONTAINS GLUTEN)', 'soy (CONTAINS SOY)'], name:popcorn chicken, category:chicken
allergens:['wheat (CONTAINS GLUTEN)'], name:hot wings, category:chicken
allergens:, name:snackbox, category:chicken
allergens:, name:crispy tenders, category:chicken
allergens:, name:original piece, category:chicken
allergens:, name:tender chicken, category:chicken
allergens:['wheat (CONTAINS GLUTEN)'], name:iced tea, category:drinks
allergens:['dairy (CONTAINS LACTOSE)'], name:pepsi, category:drinks
allergens:['wheat (CONTAINS GLUTEN)'], name:7up, category:drinks
allergens:['dairy (CONTAINS LACTOSE)'], name:fanta, category:drinks
allergens:, name:sourcy, category:drinks
allergens:, name:tropicana apple, category:drinks
allergens:, name:guava, category:drinks
allergens:, name:tea, category:drinks
allergens:, name:latte, category:drinks
allergens:, name:espresso, category:drinks
allergens:,

# 3. Experiment with Retriever

In [12]:
import weaviate
import weaviate.classes as wvc
import time
from drive_thru_bot.agent.retriever import Retriever

client = weaviate.connect_to_local()
collection = client.collections.get("Kfc_menu")

retriever = Retriever(client=client, collection_name="kfc_menu")

### Consider the following questions being asked:

- Hi, do you have cola?
- Hi I want to have a Fire Zinger Stacker without sauce and a cola
- Give me a Veggie Tender, medium, with salad
- Give me an orange chocolate milkshake, medium
- Give me the gluten free burger options
- How many calories does the Colonel have?
- Can I get a Whopper?

In [13]:
test_queries = [
    "Hi, do you have cola?",
    "Hi I want to have a Fire Zinger Stacker without sauce and a cola",
    "Give me a Veggie Tender, medium, with salad",
    "Give me an orange chocolate milkshake, medium",
    "Give me gluten free burger options",
    "How many calories does the Colonel have?",
    "Can I get a Whopper?"
]

In [14]:
for query in test_queries:
    print('Query:', query)
    init_time = time.time()
    result = retriever.query(
        query=query,
        search_type="hybrid",
        query_properties=["name", "category^3"],
        fusion_type=wvc.query.HybridFusion.RELATIVE_SCORE,
        alpha = 0.9,
        limit = 10,
        auto_limit = 1)
    
    print('Time taken: ', (time.time() - init_time)*1000, "ms")
    print([res.properties['name'] for res in result.objects], "\n\n")

Query: Hi, do you have cola?
Time taken:  18.758058547973633 ms
['Pepsi'] 


Query: Hi I want to have a Fire Zinger Stacker without sauce and a cola
Time taken:  14.547348022460938 ms
['Fire Zinger Stacker', 'Fire Zinger Stacker meal'] 


Query: Give me a Veggie Tender, medium, with salad
Time taken:  17.27581024169922 ms
['Veggie Tender', '4 Veggie Tender meal', '4 Veggie Tender meal', 'Veggie Tenders'] 


Query: Give me an orange chocolate milkshake, medium
Time taken:  13.79847526550293 ms
['Chocolate Sundae'] 


Query: Give me gluten free burger options
Time taken:  12.88461685180664 ms
['Tower Burger', 'Zinger Burger', 'Filet Burger'] 


Query: How many calories does the Colonel have?
Time taken:  9.139537811279297 ms
['Colonel Burger', 'Colonel Burger Meal', 'Colonel Stacker', 'Colonel Stacker Meal'] 


Query: Can I get a Whopper?
Time taken:  12.122154235839844 ms
['Colonel Burger', 'Colonel Stacker'] 


